## Setting elliptic curve parameter

In [2]:
from umbral import config
from umbral.curve import SECP256K1
config.set_default_curve(SECP256K1)

UmbralConfigurationError: You can only set the default curve once.  Do it once and then leave it alone.

## Generating Key-Pair for Alice

In [3]:
from umbral import keys, signing
alices_private_key = keys.UmbralPrivateKey.gen_key()
alices_public_key = alices_private_key.get_pubkey()

# alices_signing_key = keys.UmbralPrivateKey.gen_key()
# alices_verifying_key = alices_signing_key.get_pubkey()
alices_signer = signing.Signer(private_key=alices_private_key)

## Encrypting with a Alice's public key

In [4]:
from umbral import pre
plaintext = b'Proxy Re-encryption is cool!'
ciphertext, capsule = pre.encrypt(alices_public_key, plaintext)
print(ciphertext)
print(capsule)

# capsule and ciphertext are stored in IPFS

b'\xc5(t\x9d\xafH\x06\xa7\xfc\xa3\x1f\xd1\x8b\xeb:V\xb3\xf4\xa1\x15\\\xf2\xb9"\x13\x9b\x90\x86\x0b\t\xa8\xbeI\x1b\x04\x1c\xe6>\xd1\xda\x84\xff\xc0QGk\xe6jO\x00>\x94s\xa9\xc9e'
Capsule:b51927d66c49306


## Generating Key-Pair for Bob

In [5]:
bobs_private_key = keys.UmbralPrivateKey.gen_key()
bobs_public_key = bobs_private_key.get_pubkey()

# Now Bob wants access of that text from Alice

## Generating 20 Key Frags by Alice

When Alice wants to grant Bob access to open her encrypted messages, she creates threshold split re-encryption keys, or “kfrags”, which are next sent to N proxies or Ursulas.

In [6]:
kfrags = pre.generate_kfrags(delegating_privkey=alices_private_key,
                              signer=alices_signer,
                              receiving_pubkey=bobs_public_key,
                              threshold=10,
                              N=20)
kfrags

[KFrag:ce4a8f58e6d307d,
 KFrag:bbbe69f7a7f6203,
 KFrag:7f8babedb739fdb,
 KFrag:0e83fc9d3f510e9,
 KFrag:706a8e48f52173a,
 KFrag:cbfa019c54c8cc9,
 KFrag:b4a1fe5183b28d3,
 KFrag:0b1198fcd428cfd,
 KFrag:5bf43abadcec78f,
 KFrag:d0e56cd84755fde,
 KFrag:df59024a0f45df9,
 KFrag:7f2e34d7803844c,
 KFrag:089d23b6bf7c76b,
 KFrag:d76af7a9bc6d20a,
 KFrag:302ad44f6a72dc4,
 KFrag:8e7393b9cb23e99,
 KFrag:e5a488c28070fb3,
 KFrag:ab95434bf097b61,
 KFrag:43ae0d871479507,
 KFrag:f50c7d0bcc597c9]

## Ursulas perform re-encryption

- Bob asks several Ursulas to re-encrypt the capsule so he can open it - He sents the capsule to Ursulas
- Each Ursula performs re-encryption on the capsule using the kfrag provided by Alice, obtaining this way a “capsule fragment”, or cfrag.
-  Let’s mock a network or transport layer by sampling threshold random kfrags, one for each required Ursula..


In [7]:
import random
kfrags = random.sample(kfrags,  # All kfrags from above (sent by Alice)
                       10)      # M - Threshold

capsule.set_correctness_keys(delegating=alices_public_key,
                             receiving=bobs_public_key,
                             verifying=alices_public_key)

cfrags = list()
for kfrag in kfrags:
    cfrag = pre.reencrypt(kfrag=kfrag, capsule=capsule)
    cfrags.append(cfrag)
cfrags

[CFrag:921377d2f8b40f9,
 CFrag:ef5661558e9c1b7,
 CFrag:7c490404bbe2c68,
 CFrag:24dedce0d6b56d7,
 CFrag:bf5ff6539edff66,
 CFrag:8e4be01a0ae0aa8,
 CFrag:5f238e3a3fb62cd,
 CFrag:5b63b5ea9b60264,
 CFrag:362f3d43e734b80,
 CFrag:47a0d4eacd8e3c0]

## Bob activates and opens the capsule

- Bob collects the resulting cfrags from several Ursulas. Bob must gather at least threshold cfrags in order to activate the capsule.
- Bob attaches at least threshold cfrags to the capsule, which has to be prepared in advance with the necessary correctness keys.

In [8]:
capsule.set_correctness_keys(delegating=alices_public_key,
                             receiving=bobs_public_key,
                             verifying=alices_public_key)

for cfrag in cfrags:
    capsule.attach_cfrag(cfrag)

cleartext = pre.decrypt(ciphertext=ciphertext, capsule=capsule, decrypting_key=bobs_private_key)
cleartext

b'Proxy Re-encryption is cool!'